# Coseismic modelling with Pyrocko using ARIA products

**Author**: David Bekaert - Jet Propulsion Laboratory, 
            Marius Isken - Geoforschungszentrum Potsdam 

This notebook demonstrates how ARIA Geocoded UNWrapped interferogram (**GUNW**) products can be used for geophysical modelling using the [**Pyrocko**](https://pyrocko.org) package. We will demonstrate data download and manipulation using ARIA-tools, data downsampling and covariance estimation using the Pyrocko [**Kite**](https://pyrocko.org/kite/docs/current/index.html) application, and a non-linear optimisation of a finite rectangular fault plane using the Pyrocko [**Grond**](https://pyrocko.org/grond/docs/current/) application, applied to coseismic InSAR and GNSS displacements measured for the 2019 Ridgecrest Earthquake. 


**Pyrocko** is a geophysical modelling package for geodetic and sesimological data. It includes various downsampling methods and covariance estimation for InSAR data, and supports joint inversion modelling of InSAR and GNSS observations in a Bayesian framework using the Grond. A full description can be found on the [Pyrocko website](https://pyrocko.org). 

A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the [ARIA website](https://aria.jpl.nasa.gov/node/97). 




<div class="alert alert-warning">
<b>Potential errors:</b>
Both Pyrocko and ARIA-tools needs to be installed to run this notebook.
</div>

<div class="alert alert-danger">
<b>Download</b>:     
    
- The ***jupyter notebook* does not allow for interactive entering of your user-name and password, use the *jupyter terminal* instead** with the same command for interactive use.
</div>

## Download the data

In this notebook we will invert for fault slip as a result from the ridgecrest earthquake sequence, consisting of a Mw 6.4 earthquake that occured on July 4th, 2019, which was followed by a Mw 7.1 earthquake on  July 6th, 2019.

ARIA provides unwrapped and geocoded interferograms as GUNW products. Multiple coseismic interferograms with different temporal baselines spanning the ridgecrest sequence are avialble in the public archive. For this tutorial, we have pre-identified one interferometric pair for the ascending track 64 (spanning 20190710 to 20180703) and the descending track 71 (spanning 20190728 to 20190622)

<img src="support_docs/Ridgecrest.png" alt="region" width="500">
<blockquote><b>Fig. 1</b> Shakemap of the RM 7.1 - 2019 Ridgecrest Earthquake Sequence, with in red and magenta the outline of respectively the ascending and descending GUNW products that will be used in this tutorial. </blockquote>

### ARIA GUNW products

The GUNW product is an InSAR surface displacement product derived from Sentinel-1 SAR data and packaged as netCDF4 files. GUNW products contain both data and meta-data layers such as the interferometric amplitude, filtered unwrapped phase, filtered coherence, connected components, perpendicular and parallel baselines, incidence, azimuth and look angles. A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the [ARIA website](https://aria.jpl.nasa.gov/node/97).

### Download options

GUNW products are hosted at the ASF DAAC and can be downloaded from the [ARIA-products page](https://aria-products.jpl.nasa.gov) and as beta products from the [ASF DAAC data search page](https://search.asf.alaska.edu/#/). If you know the GUNW filename of the product, you can also build a download link by appending the GUNW filename to **https://<i></i>grfn.asf.alaska.edu<i></i>/door/download/** . 

Alternatively, you can use the **`ariaDownload.py`** program provided within the ARIA-tools package to download data using a command-line interface. This program wraps around the ASF DAAC API and allows for search sub-setting of GUNW products based on track number, geometry (ascending or descending), as well as spatial and temporal bounding boxes criteria. For a full description of the **`ariaDownload.py`** program, see the [ariaDownload Tutorial](https://github.com/aria-tools/ARIA-tools-docs/blob/master/JupyterDocs/ariaDownload/ariaDownload_tutorial.ipynb).

<div class="alert alert-warning">
<b>Potential download failure:</b> 
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>


Below we will use **`ariaDownload.py`** to download the pre-identified GUNW products on ascending track 64 and the descending track 71.We use the **`--track`** option to specify the track number, the  **`--ifg YYYYMMDD_YYYYMMDD`** option to specifiy the interferogram combination, and the **`--bbox`** option as **SNWE** coordinates (a string with quotation) to specify the location


<div class="alert alert-danger">
<b>Download</b>:     
    
- The ***jupyter notebook* does not allow for interactive entering of your user-name and password, use the *jupyter terminal* instead** with the same command for interactive use.
</div>

In [9]:
!ariaDownload.py  --track 64  --bbox " 35.622456  35.622456 -117.670897 -117.670897"   --ifg "20190710_20180703" -v 
!ariaDownload.py  --track 71  --bbox " 35.622456  35.622456 -117.670897 -117.670897"  --ifg "20190728_20190622" -v 

https://api.daac.asf.alaska.edu/services/search/param?asfplatform=Sentinel-1%20Interferogram%20(BETA)&output=JSON&relativeOrbit=64&bbox=-117.670897,35.622456,-117.670897,35.622456
Found: S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2
 > attempting to download https://urs.earthdata.nasa.gov/profile
 > Could not validate old cookie Jar
No existing URS cookie found, please enter Earthdata username & password:
(Credentials will not be stored, saved or logged anywhere)
 > attempting to download https://urs.earthdata.nasa.gov/profile
Username: ^C
Traceback (most recent call last):
  File "/Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaPyrocko/ascending/ASFDataDload.py", line 232, in get_new_cookie
    new_username = raw_input("Username: ")
NameError: name 'raw_input' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/dbekaert/Software/python/miniconda37/e

We can now have a look at the downloaded products:

In [10]:
!ls products

ASFDataDload.py          connectedComponents      productBoundingBox
DEM                      download_products_64.kmz unwrappedPhase
__pycache__              incidenceAngle
azimuthAngle             lookAngle
DEM                      download_products_71.kmz productBoundingBox
azimuthAngle             incidenceAngle           unwrappedPhase
connectedComponents      lookAngle


See [aria-website](https://aria.jpl.nasa.gov/node/97) for a complete overview of the filename convention. Based on the filenames, do you have an idea which product is for the ascending and descending tracks?

## Data preparation 

Now that the data has been downloaded we will prepare it for processing in the **Grond** application. Pyrocko is compatible with the output from **ARIA-tools** and can load the product using the **Kite** application. 

### Extracting data and meta-data layers

Pyrocko is compatible with the output from ARIA-tools, but requires an *unwrappedPhase, incidenceAngle, azimuthAngle, and lookAngle* file. These layers can be easily extracted leveraing the ariaExtract.py program. For a detailed overview see 

In [ ]:
# Extract into ascending/
!ariaExtract.py -w ascending -f S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2.nc -d download -l unwrappedPhase,incidenceAngle,azimuthAngle,lookAngle -sm overlap

# Extract into descending/
!ariaExtract.py -w descending -f S1-GUNW-D-R-071-tops-20190728_20190622-135213-36450N_34472N-PP-b4b2-v2_0_2.nc  -d download -l unwrappedPhase,incidenceAngle,azimuthAngle,lookAngle -sm overlap

In [ ]:
### Downsamling and covariance 

In [ ]:
Two steps are needed. First we will be using ARIA-tools to extract the required data and meta-data layers. The output of this will be geo-referenced files compatible GDAL and thus most GIS packages. **Grond** itself is expecting the data in a specific format. 

In [6]:

# now load the data
###scene.load
###scene.import_data
!conda init bash
!source activate ARIA-tools_latest

!conda activate kite
!spool --load ascending

no change     /Users/dbekaert/Software/python/miniconda37/condabin/conda
no change     /Users/dbekaert/Software/python/miniconda37/bin/conda
no change     /Users/dbekaert/Software/python/miniconda37/bin/conda-env
no change     /Users/dbekaert/Software/python/miniconda37/bin/activate
no change     /Users/dbekaert/Software/python/miniconda37/bin/deactivate
no change     /Users/dbekaert/Software/python/miniconda37/etc/profile.d/conda.sh
no change     /Users/dbekaert/Software/python/miniconda37/etc/fish/conf.d/conda.fish
no change     /Users/dbekaert/Software/python/miniconda37/shell/condabin/Conda.psm1
no change     /Users/dbekaert/Software/python/miniconda37/shell/condabin/conda-hook.ps1
no change     /Users/dbekaert/Software/python/miniconda37/lib/python3.7/site-packages/xonsh/conda.xsh
no change     /Users/dbekaert/Software/python/miniconda37/etc/profile.d/conda.csh
no change     /Users/dbekaert/.bash_profile
No action taken.

CommandNotFoundError: Your shell has not been properly conf

In [1]:
from kite import Scene

# Import Matlab container to kite
scene = Scene.import_data('descending')
scene.spool()  # start the GUI for data inspection and Quadtree parametrisation


ValueError: too many values to unpack (expected 2)

In [3]:
!ls

IntermediateFiles_2d3tfgkd
IntermediateFiles_63su6r3r
IntermediateFiles_a2jdpuvt
IntermediateFiles_pld8fxez
S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2.nc
S1-GUNW-D-R-071-tops-20190728_20190622-135213-36450N_34472N-PP-b4b2-v2_0_2.nc
ariaPyrocko_tutorial.ipynb
ascending
descending
support_docs


In [ ]:
##Green’s function download

In [ ]:
We will be using Greens fucntions to invert the surface displacement for slip at depth. Greens functions 

To download the pre-calculated Green’s functions needed in this exercise, go into you new grond project folder grond-joint-geodetic and run